Import library

In [ ]:
!pip install alibi-detect[tensorflow]

In [ ]:
from alibi_detect.od import OutlierAE, OutlierVAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image
import os
from turtle import end_fill
import cv2
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Reshape, InputLayer, Flatten
from alibi_detect.od import OutlierAE, OutlierVAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image
import random
import time
import datetime as dt
from itertools import chain

In [ ]:
def trainimgload(directory,crop_size,size,testn):
    print('load 시작')
    dataset = []  #Many ways to handle data, you can use pandas. Here, we are using a list format.
    images = os.listdir(directory)
    images.sort()
    for i, image_name in enumerate(images):
        if (image_name.split('.')[-1] == 'png'):
            image = cv2.imread(directory+ image_name)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image, 'RGB')
            if crop_size==0:
                pass
            else:
                image = image.crop(crop_size)
            image = image.resize((size, size))
            dataset.append(np.array(image))
    dataset = np.array(dataset)
    n=np.shape(dataset)[0]
    #trainn=round(n*trainratioP)
    trainn=n-testn
    if testn!=0:
        train = dataset[0:trainn]
        test = dataset[trainn:]
    else:
        train=dataset[0:n-1]
        testn=1
        test=dataset[n-1:]
    trainn=np.shape(train)[0]
    testn=np.shape(test)[0]
    train = train.astype('float32') / 255.
    test = test.astype('float32') / 255.
    print("train 갯수: ", trainn)
    print("val 갯수: ", testn)
    return train,test

이미지 로드

In [ ]:
def imgload(directoryP,crop_size,SIZE,gray):
    image_directory = directoryP
    dirname=directoryP.split('/')[-2]
    #print(dirname, 'load시작')
    images = os.listdir(image_directory)
    images.sort()
    print(images)
    imgdataset=[]
    for i, image_name in enumerate(images):
        if (image_name.split('.')[-1] == 'png'):
            image = cv2.imread(image_directory + image_name)
            if gray==1:
                image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                image = Image.fromarray(image,mode="L")
            else:
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                image = Image.fromarray(image, 'RGB')
            if crop_size==0:
                pass
            else:
                image = image.crop(crop_size)
            image = image.resize((SIZE, SIZE))
            if gray==1:
                image = np.reshape(image,(SIZE,SIZE,1))
            # image=np.array(image)
            # image[0:16,48:64,:]=0
            imgdataset.append(np.array(image))
    imgdataset = np.array(imgdataset)
    imgdatasetn=np.shape(imgdataset)
    imgdataset = imgdataset.astype('float32') / 255.
    # print(images[0])
    if imgdatasetn[0]!=1:
        print(dirname, " data number: ", imgdatasetn[0])
    return imgdataset, images

In [ ]:
crop_size=0
dir="#####빈칸을 채우세요######"
train,val=trainimgload(dir,crop_size,64,20)

In [ ]:
crop_size=0
dir="#####빈칸을 채우세요######"
test,name=imgload(dir,crop_size,64,0)

In [ ]:
plt.imshow(test[3])

모델 변수 설정

In [ ]:
size=64
#모델 변수
encoding_dim =1024
latent_dim = 1024
adam = tf.keras.optimizers.Adam(learning_rate=1e-4)
train_thresehold=99.0
ep=50
batsize=10


모델 작성

In [ ]:
dense_dim = [8, 8, 512] #Dimension of the last conv. output. This is used to work our way back in the decoder.
#Define encoder
print(train[0].shape)
encoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=train[0].shape),
        Conv2D(##, ##, strides=##, padding='same', activation=tf.nn.relu),
        Conv2D(###, ##, strides=###, padding='same', activation=tf.nn.relu),
        Conv2D(###, #, strides=###, padding='same', activation=tf.nn.relu),
        Flatten(),
        Dense(encoding_dim,)
    ])
print(encoder_net.summary())

decoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=(encoding_dim,)),
        Dense(np.prod(dense_dim)),
        Reshape(target_shape=dense_dim),
        Conv2DTranspose(###, ##, strides=##, padding='same', activation=tf.nn.relu),
        Conv2DTranspose(###, ##, strides=##, padding='same', activation=tf.nn.relu),
        Conv2DTranspose(###, ##, strides=##, padding='same', activation='sigmoid')
    ])
print(decoder_net.summary())

#latent_dim = latent_dim
# initialize outlier detector
od = OutlierVAE(threshold=.015,  # threshold for outlier score above which the element is flagged as an outlier.
                score_type='mse',  # use MSE of reconstruction error for outlier detection
                encoder_net=encoder_net,  # can also pass VAE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                latent_dim=latent_dim,
                samples=20)
print("Current threshold value is: ", od.threshold)


학습 시작

In [ ]:
od.fit(train,
       optimizer = adam,
       epochs=ep,
       batch_size=batsize,
       verbose=True)

Threshold 설정

In [ ]:
od.infer_threshold(val, outlier_type='instance', threshold_perc=train_thresehold)
print("Current threshold value is: ", od.threshold)

X = val
od_preds = od.predict(X,
                      outlier_type='instance',    # use 'feature' or 'instance' level
                      return_feature_score=True,  # scores used to determine outliers
                      return_instance_score=True)
#Scatter plot of instance scores. using the built-in function for the scatterplot.
target = np.zeros(X.shape[0],).astype(int)  # Ground truth (all ones for bad images)
labels = ['normal', 'outlier']
plot_instance_score(od_preds, target, labels, od.threshold) #pred, target, labels, threshold

Model save

In [ ]:
from alibi_detect.utils.saving import save_detector
filepath="#####빈칸을 채우세요######"
save_detector(od, filepath)

Model load

In [ ]:
filepath="#####빈칸을 채우세요######"
from alibi_detect.utils.saving import load_detector
od = load_detector(filepath)
print("Current threshold value is: ", od.threshold)

In [ ]:
a=0
X = test[5*a:5*a+5]
od_preds = od.predict(X,
                      outlier_type='instance',    # use 'feature' or 'instance' level
                      return_feature_score=True,  # scores used to determine outliers
                      return_instance_score=True)
X_recon = od.vae(X).numpy()
plot_feature_outlier_image(od_preds,
                           X,
                           X_recon=X_recon,
                           max_instances=5,  # max nb of instances to display
                           outliers_only=False,
                           n_channels=3)  # only show outlier predictions
ascore = od_preds['data']['instance_score']
print(name[5*a:5*a+5])
print(ascore)
target = np.zeros(X.shape[0],).astype(int)  # Ground truth (all ones for bad images)
labels = ['normal', 'outlier']
plot_instance_score(od_preds, target, labels, od.threshold)
print("Current threshold value is: ", od.threshold)
